In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr
import math

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import nibabel as nib

In [5]:
from nilearn import plotting


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [6]:
data_path = '/gpfs/projects/sanlab/bsmith16/data/'

## Setup

In [7]:
metadata = pickle.load(open(data_path + "first_subs_nifti_metadata.pkl",'rb'))

In [47]:
from measure_results import *

## Full analysis with nested CV

In [10]:
loocv_results = pickle.load(open(data_path + "cv_train_test_60subjs_sets_outer_n_loocv.pkl",'rb'))
test_scores_same=loocv_results[0]
tt_results=loocv_results[1]
results_by_trainset_item=loocv_results[2]

FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/projects/sanlab/bsmith16/data/cv_train_test_60subjs_sets_outer_n_loocv.pkl'

In [21]:
results = pd.concat([metadata,results_by_trainset_item],axis=1)

In [22]:
results['result_group'] = results.group + "_" + results.run

In [23]:


within_subj_predictions = within_group_performance(results,pearsonr_cor_func,'result_group')
    
r2_score_within_subj = math.pow(np.mean(within_subj_predictions),2)

In [24]:
math.pow(pearsonr(results['y'],results['y_pred'])[0],2)

0.22593644995643092

In [25]:
r2_score_within_subj

0.24964975312247112

In [26]:

within_subj_predictions_spearman = within_group_performance(results,spearmanr_cor_func,'result_group')
    
rho_score_within_subj = np.mean(within_subj_predictions)

print(spearmanr(results['y'],results['y_pred'])[0])
print(rho_score_within_subj)

0.4765406183765473
0.499649630363589


OK great that appears to have worked??? No improvement over previous, but that's OK; we're just trying to get a more consistent measure working.

In [27]:
len(tt_results)

60

In [28]:
len(loocv_results)

3

## simple

Doesn't seem to be able to give us the model for each individual fold so we might leave this and just focus on the method above.

In [12]:
loocv_simple_scores = pickle.load(open(data_path + "train_test_simple_results_ns_w_hrf_from_spm_40subs.pkl",'rb'))

len(loocv_simple_scores)



6

OK we can line up the obs, train_y_pred, and the metadata with a simple concat.

In [39]:
if np.all(loocv_simple_scores['obs']==loocv_simple_scores['metadata'].response):
    results = pd.concat([loocv_simple_scores['metadata'],
                         pd.Series(loocv_simple_scores['train_y_pred'],name='train_y_pred')
                        ],axis=1)
else:
    raise Exception("error: obs out of sync with metadata")

In [42]:
results.columns

Index(['beta', 'type', 'task', 'event_id', 'isi_pre', 'onset', 'duration',
       'food_pic', 'food_num', 'cond', 'health_cond', 'liking_cond',
       'liking_rating', 'response', 'isi_post', 'end', 'run', 'wave',
       'subject', 'train_y_pred'],
      dtype='object')

In [43]:
results['result_group'] = results.subject + "_" + results.run

In [51]:
within_group_predictions = within_group_performance(
    results,pearsonr_cor_func,'result_group',obs_col_name='response',pred_col_name='train_y_pred') 
r2_score_within_subj = math.pow(np.mean(within_group_predictions),2)
print("Pearson's R^2 within subject and run:")
print(r2_score_within_subj)

within_subj_predictions = within_group_performance(results,pearsonr_cor_func,'subject',obs_col_name='response',pred_col_name='train_y_pred') 
r2_score_within_subj_run = math.pow(np.mean(within_subj_predictions),2)
print("Pearson's R^2 within subject only:")
print(r2_score_within_subj_run)



print("Pearson's R^2 across subjects:")
math.pow(pearsonr(results['response'],results['train_y_pred'])[0],2)

Pearson's R^2 within subject and run:
0.9732283507257504
Pearson's R^2 within subject only:
0.9732481125298129
Pearson's R^2 across subjects:


0.9776390624681254

I can't see how to get a cross-validated list out of this object. So I think we will not use this method again--I can't properly ensure a cross-validated accuracy score.

## No HRF

In [58]:
loocv_simple_scores = pickle.load(open(data_path + "train_test_simple_results_ns_no_hrf_from_spm_40subs.pkl",'rb'))

len(loocv_simple_scores)



6

In [70]:
def print_r_squared_results(loocv_simple_scores):
    if np.all(loocv_simple_scores['obs']==loocv_simple_scores['metadata'].response):
        results = pd.concat([loocv_simple_scores['metadata'],
                             pd.Series(loocv_simple_scores['train_y_pred'],name='train_y_pred')
                            ],axis=1)
    else:
        raise Exception("error: obs out of sync with metadata")

    results['result_group'] = results.subject + "_" + results.run

    within_group_predictions = within_group_performance(
        results,pearsonr_cor_func,'result_group',obs_col_name='response',pred_col_name='train_y_pred') 
    r2_score_within_subj = math.pow(np.mean(within_group_predictions),2)
    print("Pearson's R^2 within subject and run:")
    print(r2_score_within_subj)

    within_subj_predictions = within_group_performance(results,pearsonr_cor_func,'subject',obs_col_name='response',pred_col_name='train_y_pred') 
    r2_score_within_subj_run = math.pow(np.mean(within_subj_predictions),2)
    print("Pearson's R^2 within subject only:")
    print(r2_score_within_subj_run)



    print("Pearson's R^2 across subjects:")
    print(math.pow(pearsonr(results['response'],results['train_y_pred'])[0],2))

In [64]:
print_r_squared_results(loocv_simple_scores)

Pearson's R^2 within subject and run:
0.9876036107669584
Pearson's R^2 within subject only:
0.9873310369883388
Pearson's R^2 across subjects:


## ns_6s_no_hrf_from_spm_40subs

In [65]:
loocv_simple_scores = pickle.load(open(data_path + "train_test_simple_results_ns_6s_no_hrf_from_spm_40subs.pkl",'rb'))

len(loocv_simple_scores)



6

In [66]:
print_r_squared_results(loocv_simple_scores)

Pearson's R^2 within subject and run:
0.982710128912157
Pearson's R^2 within subject only:
0.9826215680586369
Pearson's R^2 across subjects:


## ns_w_hrf_from_spm_at_fingerpress_40subs

In [69]:
loocv_simple_scores = pickle.load(
    open(data_path + 
         "train_test_simple_results_ns_w_hrf_from_spm_at_fingerpress_40subs.pkl",'rb'))

len(loocv_simple_scores)



6

In [71]:
print_r_squared_results(loocv_simple_scores)

Pearson's R^2 within subject and run:
0.96874064080543
Pearson's R^2 within subject only:
0.9672889124896762
Pearson's R^2 across subjects:
0.9710799807879283


## ns_2s_split_0_40subs

In [73]:
loocv_simple_scores = pickle.load(
    open(data_path + 
         "train_test_simple_results_ns_2s_split_0_40subs.pkl",'rb'))

len(loocv_simple_scores)



6

In [74]:
print_r_squared_results(loocv_simple_scores)

Pearson's R^2 within subject and run:
0.9876036107669584
Pearson's R^2 within subject only:
0.9873310369883388
Pearson's R^2 across subjects:
0.989454130644467
